<a href="https://colab.research.google.com/github/Kushagra-1/Extract-prices-to-csv/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import json

def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)

    else:
        print("Did not receieve OK response from Coinbase API")


if __name__ == "__main__":
    # we set which pair we want to retrieve data for
    pair = "ETH/USD"
    fetch_daily_data(symbol=pair)

In [2]:
df = pd.read_csv('/content/Coinbase_ETHUSD_dailydata.csv')
df.head()

,unix,low,high,open,close,volume,date,vol_fiat
0,1633392000,3365.34,3476.03,3386.90,3424.72,84574.930123,2021-10-05,2.896455e+08
1,1633305600,3269.67,3438.91,3419.63,3387.20,163829.626778,2021-10-04,5.549237e+08
2,1633219200,3343.36,3489.18,3388.67,3420.07,124271.048646,2021-10-03,4.250157e+08
3,1633132800,3258.94,3469.95,3311.20,3387.82,144778.384500,2021-10-02,4.904831e+08
4,1633046400,2967.41,3337.15,3001.14,3311.16,201035.513451,2021-10-01,6.656608e+08
